In [2]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification

model_name="openai/clip-vit-base-patch16" 
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)


model_name="openai/clip-vit-base-patch16"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForZeroShotImageClassification.from_pretrained(
    model_name,
    device_map="auto",
)
model_4bit = AutoModelForZeroShotImageClassification.from_pretrained(
    model_name,
    config=quantization_config,
    device_map="auto",
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
model.eval()


dtype=list(model.parameters())[0].dtype
print("原始精度:",dtype)
dest_dtype=list(model_4bit.parameters())[0].dtype
print("量化精度:",dest_dtype)

# 检查模型的量化配置
print("量化配置:", model_4bit.config.quantization_config)

def print_model_info(model):
    total_params = 0
    for name, param in model.named_parameters():
        total_params += param.numel()
    #print(f"Total parameters: {total_params / 1e6}M")
    return total_params

total_model_size=print_model_info(model)
total_model_4bit_size=print_model_info(model_4bit)
print("模型参数个数：",total_model_size)
print("量化后的模型参数个数：",total_model_4bit_size)

dtype_to_bytes = {
    torch.float32: 4,  # FP32: 4字节
    torch.float16: 2,  # FP16: 2字节
    torch.int8: 1,     # INT8: 1字节
    torch.int32: 4,    # INT32: 4字节
    torch.int64: 8,    # INT64: 8字节
    torch.float64: 8,  # FP64 (double): 8字节
}
model_size = total_model_size * dtype_to_bytes[dtype]
model_size = total_model_size * dtype_to_bytes[dtype]
print(f'origin Model size: {model_size / (1024**2):.2f} MB')
model_size = total_model_4bit_size * dtype_to_bytes[dest_dtype]
print(f'quan Model size: {model_size / (1024**2):.2f} MB')

model_4bit.save_pretrained("./tmp")
# model.save_pretrained("/tmp/o")


PackageNotFoundError: No package metadata was found for bitsandbytes